Here we test how sensitive dft based free energy calculations are to changes in integral grid size and optimisation tightness

In [50]:
import tarfile
import copy
import numpy as np
from cclib.parser.utils import convertor
from ase import Atoms
from ml_utils import extended_xyz_parse
from gausspy import Gaussian
from ase_extensions.ase_utils import *
from cc_notebook import *

In [17]:
cd free_energy_test

/home/clyde/Dropbox/Project Stuff/Notebooks/test_cases/free_energy_test


In [23]:
#set of 19 atom molecules computed at the g4mp2 level of theory from Anatole's 134K ab inition calculations paper
with tarfile.open('../../ML/134k_molecules/dsC7O2H10nsd.xyz.tar.bz2','r:bz2') as data_tar:
    mol_f=data_tar.extractfile(data_tar.next())
    xyz_data = mol_f.read()
    mol_f.close()
    mol_data = extended_xyz_parse(xyz_data)
mol = Atoms(symbols=mol_data['symbols'], positions=mol_data['coords'])

In [33]:
mol1 = copy.deepcopy(mol)
mol1.set_calculator(Gaussian(label='test_free_energy1',method='b3lyp',basis='6-31G(2df,p)',opt='opt'))
mol1.calc.set_job(nodes=8, memory=8*2000, queue='pqmb',version='g09-d01')
mol1.calc.restart(opt='', freq='freq')
#mol1.calc.start()

In [37]:
mol2 = copy.deepcopy(mol)
mol2.set_calculator(Gaussian(label='test_free_energy2',method='b3lyp',basis='6-31G(2df,p)',opt='tight'))
mol2.calc.set_job(nodes=8, memory=8*2000, queue='pqmb',version='g09-d01')
mol2.calc.restart(opt='', freq='freq')
#mol2.calc.start()

In [39]:
mol3 = copy.deepcopy(mol)
mol3.set_calculator(Gaussian(label='test_free_energy3',method='b3lyp',basis='6-31G(2df,p)',opt='tight',integral='ultrafine'))
mol3.calc.set_job(nodes=8, memory=8*2000, queue='pqmb',version='g09-d01')
mol3.calc.restart(opt='', freq='freq')
#mol3.calc.start()

In [47]:
check_calcs([mol1,mol2,mol3])
mols_to_html([mol1,mol2,mol3])

test_free_energy1_restart_1,,Success
test_free_energy2_restart_1,,Success
test_free_energy3_restart_1,,Success


In [52]:
free_energies = np.array([mol1.calc.max_data['freeenergy'], mol2.calc.max_data['freeenergy'], mol3.calc.max_data['freeenergy']])
convertor(free_energies, 'hartree', 'kcal')

array([-265495.81724668, -265495.81536415, -265495.83544445])

# Negligible difference